# Phase 4: Integrate Generative AI  
In this phase, Generative AI was integrated into the system to generate personalized, data-driven sleep recommendations. This stage focuses on implementing two prompt structures and conducting templates comparison and analysis to evaluate how each template interprets the student’s sleep habits and produces tailored insights.

## 1. Implementation
In this section, two generative AI prompt templates are implemented to enhance the system’s ability to analyze and interpret student sleep patterns. Both templates interact with the LLaMA model through the Groq API.
### 1.1 Template 1: Brief Recommendation
This template generates a short, focused recommendation based on the student’s sleep habits. It identifies the main issue and provides one or two essential suggestions. The output is intentionally concise and easy to understand, making it suitable for quick feedback.
### 1.2 Template 2: Detailed Recommendation
This template offers a comprehensive evaluation of the student’s sleep behavior. It compares the user’s inputs to dataset averages, analyzes the impact of habits such as screen time, caffeine, and physical activity, and examines weekday/weekend schedule consistency. The generated recommendation includes an overview, detailed analysis, and a multi-step action plan tailored for meaningful improvement.
### 1.3 Code of Templates 1 & 2  

In [ ]:
!pip install groq

In [32]:
import os
import pandas as pd
from groq import Groq

# ==================================================================
# Configuration.
# ==================================================================
DATA_PATH = r"../Dataset/student_sleep_patterns_preprocessed.csv"   
GROQ_API_KEY = "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"  
def load_dataset(path: str) -> pd.DataFrame:
    """Load sleep dataset."""
    df = pd.read_csv(path)
    return df

# ==================================================================
# This function computes descriptive statistics for all major attributes in the dataset.
# The extracted statistics (averages, distributions, and dominant sleep patterns) are later
# used as contextual information inside the prompt templates, enabling the model to compare
# the user’s inputs against typical behaviors observed in the dataset.
# ==================================================================
def compute_dataset_stats(df: pd.DataFrame) -> dict:
    """
    Compute descriptive statistics for all main attributes in the dataset.
    These stats will be used as context when generating recommendations.
    """
    stats = {
        # Demographic and academic attributes
        "avg_age": df["Age"].mean(),
        "gender_distribution": df["Gender"].value_counts(normalize=True).to_dict(),
        "year_distribution": df["University_Year"].value_counts(normalize=True).to_dict(),

        # Lifestyle / habit attributes
        "avg_sleep": df["Sleep_Duration"].mean(),
        "avg_study": df["Study_Hours"].mean(),
        "avg_screen": df["Screen_Time"].mean(),
        "avg_caffeine": df["Caffeine_Intake"].mean(),
        "avg_activity": df["Physical_Activity"].mean(),

        # Sleep quality and schedule patterns 
        "sleep_quality_distribution": df["Sleep_Quality"].value_counts(normalize=True).to_dict(),
        "weekday_start_mode": df["Weekday_Sleep_Start"].mode()[0],
        "weekend_start_mode": df["Weekend_Sleep_Start"].mode()[0],
        "weekday_end_mode": df["Weekday_Sleep_End"].mode()[0],
        "weekend_end_mode": df["Weekend_Sleep_End"].mode()[0],
    }
    return stats

# ==================================================================
# This function gathers all necessary inputs from the user.
# ==================================================================
def get_user_input() -> dict:
   
    print("=" * 60)
    print("STUDENT SLEEP PROFILE – USER INPUT".center(60))
    print("=" * 60)

    age = int(input("Age (years): "))
    gender = input("Gender (0 = Other, 1 = Male, 2 = Female): ").strip()
    uni_year = int(input("University year (1–4): "))

    sleep_duration = float(input("Average sleep duration (hours per night): "))
    study_hours = float(input("Average study hours per day: "))
    screen_time = float(input("Average screen time per day (hours): "))

    caffeine = float(input("Caffeine intake per day (cups): "))
    activity = float(input("Physical activity per week (minutes): "))

    weekday_start = input("Weekday sleep start (Early / Medium / Late): ").strip()
    weekend_start = input("Weekend sleep start (Early / Medium / Late): ").strip()
    weekday_end = input("Weekday wake-up time (Early / Medium / Late): ").strip()
    weekend_end = input("Weekend wake-up time (Early / Medium / Late): ").strip()

    user_profile = {
        "Age": age,
        "Gender": gender,
        "University_Year": uni_year,
        "Sleep_Duration": sleep_duration,
        "Study_Hours": study_hours,
        "Screen_Time": screen_time,
        "Caffeine_Intake": caffeine,
        "Physical_Activity": activity,
        "Weekday_Sleep_Start": weekday_start,
        "Weekend_Sleep_Start": weekend_start,
        "Weekday_Sleep_End": weekday_end,
        "Weekend_Sleep_End": weekend_end,
    }

    return user_profile

# ==================================================================
# Template 1: Brief Recommendation (short, focused feedback) 
# ==================================================================
def build_brief_prompt(user: dict, stats: dict) -> str:
    """Prompt for Template 1: Brief Recommendation."""
    prompt = f"""
You are a sleep coach for university students.

Student profile:
- Age: {user["Age"]} (dataset average age: {stats["avg_age"]:.1f})
- Gender (dataset coded as 0=Other, 1=Male, 2=Female): {user["Gender"]}
- University year: {user["University_Year"]}
- Sleep duration: {user["Sleep_Duration"]:.1f} hours/night (dataset average: {stats["avg_sleep"]:.1f})
- Study hours: {user["Study_Hours"]:.1f} hours/day (dataset average: {stats["avg_study"]:.1f})
- Screen time: {user["Screen_Time"]:.1f} hours/day (dataset average: {stats["avg_screen"]:.1f})
- Caffeine intake: {user["Caffeine_Intake"]:.1f} cups/day (dataset average: {stats["avg_caffeine"]:.1f})
- Physical activity: {user["Physical_Activity"]:.0f} minutes/week (dataset average: {stats["avg_activity"]:.0f})
- Weekday sleep: {user["Weekday_Sleep_Start"]} → {user["Weekday_Sleep_End"]}
- Weekend sleep: {user["Weekend_Sleep_Start"]} → {user["Weekend_Sleep_End"]}

Task:
Give a very brief sleep recommendation (max 3–4 sentences).
Focus only on:
- The main problem in this sleep pattern.
- 1 or 2 key, practical suggestions.

Write in clear, simple English for a university student.
"""
    return prompt

# =========================================================================
# Template 2: Detailed Recommendation (full analysis & action plan) 
# =========================================================================
def build_detailed_prompt(user: dict, stats: dict) -> str:
    """Prompt for Template 2: Detailed Recommendation."""
    prompt = f"""
You are an expert sleep coach for university students.

You are given this student profile:

Demographic and academic:
- Age: {user["Age"]} (dataset average age: {stats["avg_age"]:.1f})
- Gender (0=Other, 1=Male, 2=Female): {user["Gender"]}
- University year: {user["University_Year"]}

Habits (with dataset comparison):
- Sleep duration: {user["Sleep_Duration"]:.1f} hours/night 
  (dataset average: {stats["avg_sleep"]:.1f} hours)
- Study hours: {user["Study_Hours"]:.1f} hours/day 
  (dataset average: {stats["avg_study"]:.1f} hours)
- Screen time: {user["Screen_Time"]:.1f} hours/day 
  (dataset average: {stats["avg_screen"]:.1f} hours)
- Caffeine intake: {user["Caffeine_Intake"]:.1f} cups/day 
  (dataset average: {stats["avg_caffeine"]:.1f} cups)
- Physical activity: {user["Physical_Activity"]:.0f} minutes/week 
  (dataset average: {stats["avg_activity"]:.0f} minutes)

Sleep schedule patterns (dataset modes in brackets):
- Weekday: {user["Weekday_Sleep_Start"]} sleep → {user["Weekday_Sleep_End"]} wake 
  [most common pattern in dataset: {stats["weekday_start_mode"]} → {stats["weekday_end_mode"]}]
- Weekend: {user["Weekend_Sleep_Start"]} sleep → {user["Weekend_Sleep_End"]} wake 
  [most common pattern in dataset: {stats["weekend_start_mode"]} → {stats["weekend_end_mode"]}]

Task:
Write a detailed sleep recommendation for this student. Structure your answer as:

1. Short overview (2–3 sentences) describing the main issues in this pattern.
2. Analysis:
   - Sleep duration vs dataset average.
   - Study hours and screen time impact.
   - Caffeine and physical activity impact.
   - Weekday vs weekend schedule consistency.
3. Action plan:
   - 3–5 concrete steps to improve sleep quality (with specific numbers when possible, e.g., "reduce screen time to X hours", "aim for Y minutes of activity").
4. Final encouragement message in one sentence.

Write in clear, friendly English, suitable for a university student.
"""
    return prompt


# =========================================================================
# Send a prompt to the Groq LLaMA model and return the generated response.
# =========================================================================
def call_groq(prompt: str, model_name: str = "llama-3.1-8b-instant") -> str:
    client = Groq(api_key=GROQ_API_KEY)

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful sleep coach for university students."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.7,
        max_tokens=900,
    )

    return response.choices[0].message.content

# =========================================================================
# Print the collected user profile so it remains visible in the output cell.
# =========================================================================
def pretty_print_profile(user_profile: dict):
  
    print("\n" + "=" * 60)
    print("YOUR ENTERED SLEEP PROFILE".center(60))
    print("=" * 60)
    for key, value in user_profile.items():
        print(f"{key}: {value}")

# =========================================================================
# Main Function
# =========================================================================
def main():
    # Load dataset and compute stats once
    df = load_dataset(DATA_PATH)
    stats = compute_dataset_stats(df)

    # Get user profile from input
    user_profile = get_user_input()

    # Show the filled inputs in the same output cell
    pretty_print_profile(user_profile)

# =========================================================================
# Generate Template 1: Brief Recommendation
# =========================================================================
    brief_prompt = build_brief_prompt(user_profile, stats)
    print("\n" + "=" * 60)
    print("TEMPLATE 1: BRIEF RECOMMENDATION".center(60))
    print("=" * 60)
    brief_answer = call_groq(brief_prompt)
    print(brief_answer)

# =========================================================================
# Generate Template 2: Detailed Recommendation
# =========================================================================
    detailed_prompt = build_detailed_prompt(user_profile, stats)
    print("\n" + "=" * 60)
    print("TEMPLATE 2: DETAILED RECOMMENDATION".center(60))
    print("=" * 60)
    detailed_answer = call_groq(detailed_prompt)
    print(detailed_answer)


if __name__ == "__main__":
    main()


             STUDENT SLEEP PROFILE – USER INPUT             

                 YOUR ENTERED SLEEP PROFILE                 
Age: 24
Gender: 2
University_Year: 4
Sleep_Duration: 5.0
Study_Hours: 4.0
Screen_Time: 7.0
Caffeine_Intake: 3.0
Physical_Activity: 140.0
Weekday_Sleep_Start: Late
Weekend_Sleep_Start: Late
Weekday_Sleep_End: Early
Weekend_Sleep_End: Late

              TEMPLATE 1: BRIEF RECOMMENDATION              
Based on your profile, I'd say your main sleep problem is that you're not getting enough sleep, especially considering you're in your 4th year of university and need to perform well. To improve your sleep, try these two things:

1. **Limit your screen time**: You have a lot of screen time, which can keep your brain active and make it hard to fall asleep. Try to reduce your screen time by at least 2 hours before bedtime.
2. **Go to bed earlier on weekdays**: Your current sleep schedule is late → early, which can be tough on your body. Try to adjust your sleep schedule to 

 
## 2. Template Comparison & Analysis


## 3. Justification
